In [ ]:
import time
import random

import param
import numpy as np
import pandas as pd
import panel as pn
pn.extension()

## Dev experience

### Bind on reference value, not value

#### Good

Be sure to bind on `param.{parameter}`, not just `{parameter}`.

In [ ]:
def show_clicks(clicks):
    return f"Number of clicks: {clicks}"

button = pn.widgets.Button(name="Click me!")
clicks = pn.bind(show_clicks, button.param.clicks)
pn.Row(button, clicks)

#### Bad

If only on `{parameter}`, it will not trigger an update on change.

In [ ]:
def show_clicks(clicks):
    return f"Number of clicks: {clicks}"

button = pn.widgets.Button(name="Click me!")
clicks = pn.bind(show_clicks, button.clicks)  # not button.clicks!
pn.Row(button, clicks)

### Inherit from `pn.viewer.Viewer`

Instead of inheriting from `param.Parameterized`, opting for inheritance from `pn.viewable.Viewer` allows direct invocation of the class, resembling a native Panel object, i.e. `ExampleApp().servable()` vs `ExampleApp().view().servable()`.

#### Good

In [ ]:
class ExampleApp(pn.viewable.Viewer):

    ...

    def __panel__(self):
        return pn.template.FastListTemplate(
            main=[...],
            sidebar=[...],
        )

ExampleApp().servable();  # semi-colon to suppress output in notebook

#### Okay

In [ ]:
class ExampleApp(param.Parameterized):

    ...

    def view(self):
        return pn.template.FastListTemplate(
            main=[...],
            sidebar=[...],
        )

ExampleApp().view().servable();  # semi-colon to suppress output in notebook

### Build widgets from parameters

To translate parameters into widgets, use `pn.Param`.


#### Good

In [ ]:
class ExampleApp(pn.viewable.Viewer):

    width = param.Integer(default=100, bounds=(1, 200), label="Width of box")
    height = param.Integer(default=100, bounds=(1, 250), label="Height of box")
    color = param.Color(default="red", label="Color of box")

    def __panel__(self):
        return pn.Column(
            pn.Param(self, widgets={"height": pn.widgets.IntInput}),
            pn.pane.HTML(
                width=self.param.width,
                height=self.param.height,
                styles={"background-color": self.param.color},
            ),
        )


ExampleApp()

#### Good

You can also use `from_param` to manually build each component.

In [ ]:
class ExampleApp(pn.viewable.Viewer):

    width = param.Integer(default=100, bounds=(1, 200), label="Width of box")
    height = param.Integer(default=100, bounds=(1, 250), label="Height of box")
    color = param.Color(default="red", label="Color of box")

    def __panel__(self):
        width_slider = pn.widgets.IntSlider.from_param(self.param.width)
        height_input = pn.widgets.IntInput.from_param(self.param.height)
        color_picker = pn.widgets.ColorPicker.from_param(self.param.color)
        return pn.Column(
            width_slider,
            height_input,
            color_picker,
            pn.pane.HTML(
                width=self.param.width,
                height=self.param.height,
                styles={"background-color": self.param.color},
            ),
        )


ExampleApp()

#### Bad

If you instantiate individually through `param`, it's not bidirectional.


In [ ]:
class ExampleApp(pn.viewable.Viewer):

    width = param.Integer(default=100, bounds=(1, 200), label="Width of box")
    height = param.Integer(default=100, bounds=(1, 250), label="Height of box")
    color = param.Color(default="red", label="Color of box")

    def __panel__(self):
        width_slider = pn.widgets.IntSlider(
            value=self.param.width,
            start=self.param["width"].bounds[0],
            end=self.param["width"].bounds[1],
            name=self.param["width"].label,
        )
        height_input = pn.widgets.IntInput(
            value=self.param.height,
            start=self.param["height"].bounds[0],
            end=self.param["height"].bounds[1],
            name=self.param["height"].label,
        )
        color_picker = pn.widgets.ColorPicker(
            value=self.param.color,
            name=self.param["color"].label,
            width=200,
        )
        return pn.Column(
            width_slider,
            height_input,
            color_picker,
            pn.pane.HTML(
                width=self.param.width,
                height=self.param.height,
                styles={"background-color": self.param.color},
            ),
        )


ExampleApp()

#### Bad

It's possible to `link` each widget to `self` with `bidirectional=True`, but certain keyword arguments, like `bounds`, cannot be linked easily.

In [ ]:
class ExampleApp(pn.viewable.Viewer):

    width = param.Integer(default=100, bounds=(1, 200), label="Width of box")
    height = param.Integer(default=100, bounds=(1, 250), label="Height of box")
    color = param.Color(default="red", label="Color of box")

    def __panel__(self):
        width_slider = pn.widgets.IntSlider()
        height_input = pn.widgets.IntInput()
        color_picker = pn.widgets.ColorPicker()

        width_slider.link(self, value="width", bidirectional=True)
        height_input.link(self, value="height", bidirectional=True)
        color_picker.link(self, value="color", bidirectional=True)

        return pn.Column(
            width_slider,
            height_input,
            color_picker,
            pn.pane.HTML(
                width=self.param.width,
                height=self.param.height,
                styles={"background-color": self.param.color},
            ),
        )


ExampleApp()

#### Bad

Widgets should not be used as parameters.

In [ ]:
class ExampleApp(pn.viewable.Viewer):

    width = pn.widgets.IntSlider()
    height = pn.widgets.IntInput()
    color = pn.widgets.ColorPicker()


### Show templates in notebooks

Templates, at the time of writing, are not able to be rendered properly in Jupyter notebooks.

To continue working with templates in notebooks, call `show` to pop up a new browser window.

#### Okay

In [ ]:
template = pn.template.FastListTemplate(
    main=[...],
    sidebar=[...],
)

# template.show()  # commented out to disable opening a new browser tab in example

#### Okay

Alternatively, you can use a barebones notebook template like the one below.

In [ ]:
class NotebookPlaceholderTemplate(pn.viewable.Viewer):
    main = param.List()
    sidebar = param.List()
    header = param.List()
    title = param.String()

    def __panel__(self):
        title = pn.pane.Markdown(f"# {self.title}", sizing_mode="stretch_width")
        # pastel blue
        header_row = pn.Row(
            title,
            *self.header,
            sizing_mode="stretch_width",
            styles={"background": "#e6f2ff"},
        )
        main_col = pn.WidgetBox(*self.main, sizing_mode="stretch_both")
        sidebar_col = pn.WidgetBox(
            *self.sidebar, width=300, sizing_mode="stretch_height"
        )
        return pn.Column(
            header_row,
            pn.Row(sidebar_col, main_col, sizing_mode="stretch_both"),
            sizing_mode="stretch_both",
            min_height=400,
        )

template = pn.template.FastListTemplate(
    main=[...],
    sidebar=[...],
)
template

### Yield to show intermediate values

#### Good

Use a generator (yield) to provide incremental updates.

In [ ]:
def increment_to_value(value):
    for i in range(value):
        time.sleep(0.1)
        yield i

slider = pn.widgets.IntSlider(start=1, end=10)
output = pn.bind(increment_to_value, slider.param.value_throttled)
pn.Row(slider, output)

### Watch side effects

#### Good

For functions that trigger side effects, i.e. do not return anything (or returns None), be sure to set `watch=True` on `pn.bind` or `pn.depends`.

In [ ]:
def print_clicks(clicks):
    print(f"Number of clicks: {clicks}")

button = pn.widgets.Button(name="Click me!")
pn.bind(print_clicks, button.param.clicks, watch=True)
button

#### Okay

For buttons, you can also use `on_click`.

In [ ]:
def print_clicks(event):
    clicks = event.new
    print(f"Number of clicks: {clicks}")

button = pn.widgets.Button(name="Click me!", on_click=print_clicks)
button

#### Okay

For all other widgets, use `obj.param.watch()` for side effects.

In [ ]:
def print_clicks(event):
    clicks = event.new
    print(f"Number of clicks: {clicks}")

button = pn.widgets.Button(name="Click me!")
button.param.watch(print_clicks, "clicks")
button

## User Experience

### Update params effectively

#### Good

Use `obj.param.update`:
- to update multiple parameters on an object simultaneously
- as a context manager to temporarily set values, restoring original values on completion

In [ ]:
def run(event):
    with progress.param.update(
        bar_color="primary",
        active=True,
    ):
        for i in range(0, 101):
            time.sleep(0.01)
            progress.value = i

button = pn.widgets.Button(name="Run", on_click=run)
progress = pn.indicators.Progress(value=100, active=False, bar_color="dark")
pn.Row(button, progress)

#### Bad

The following shows setting parameters individually, which could be inefficient.

In [ ]:
def run(event):
    try:
        progress.bar_color = "primary"
        progress.active = True
        for i in range(0, 101):
            time.sleep(0.01)
            progress.value = i
    finally:
        progress.bar_color = "dark"
        progress.active = False

button = pn.widgets.Button(name="Run", on_click=run)
progress = pn.indicators.Progress(value=100, active=False, bar_color="dark")
pn.Row(button, progress)

#### Good

Use `batch_call_watchers` to update multiple params *on separate objects* simultaneously.

In [ ]:
class ExampleBar(pn.viewable.Viewer):

    bar_color = "dark"
    active = False

    def __init__(self):
        self.progress = pn.indicators.Progress(
            value=100, active=self.param.active, bar_color=self.param.bar_color
        )

    def _process(self, event):
        with param.parameterized.batch_call_watchers(self):
            self._text_1.value = "Hey"
            self._text_2.value = "Hey"

    def __panel__(self):
        return pn.Row(self._button, self._text_1, self._text_2)


ExampleApp()

### Throttle slider callbacks

#### Good

To prevent sliders from triggering excessive callbacks, set `throttled=True` so that it only triggers once upon mouse-up.

In [ ]:
pn.extension(throttled=True)

def callback(value):
    time.sleep(2)
    return f"# {value}"

slider = pn.widgets.IntSlider(end=10)
output = pn.bind(callback, slider)
pn.Row(slider, output)

#### Good

Alternatively, limit the scope by binding against `value_throttled` instead of `value`.

In [ ]:
def callback(value):
    time.sleep(2)
    return f"# {value}"

slider = pn.widgets.IntSlider(end=10)
output = pn.bind(callback, slider.param.value_throttled)
pn.Row(slider, output)

#### Bad

If the operation is expensive, binding against `value` can be really slow.

In [ ]:
def callback(value):
    time.sleep(2)
    return f"# {value}"

slider = pn.widgets.IntSlider(end=10)
output = pn.bind(callback, slider.param.value)
pn.Row(slider, output)

### Defer expensive operations

#### Good

Its easy defer the execution of all bound and displayed functions with `pn.extension(defer_load=True)`.

In [ ]:
pn.extension(defer_load=True, loading_indicator=True)

def onload():
    time.sleep(5)  # simulate expensive operations
    return pn.Column(
        "Welcome to this app!",
    )

layout = pn.Column("Check this out!", onload)
# layout.show()

#### Okay

If you need finer control, start by instantiating the initial layout with placeholder `pn.Columns`, then populate it later in `onload`.

In [ ]:
import time

def onload():
    time.sleep(1)  # simulate expensive operations
    layout.objects = [
        "Welcome to this app!",
    ]

layout = pn.Column("Loading...")
display(layout)
pn.state.onload(onload)

### Show indicator while computing

#### Good

Set `loading=True` to show a spinner while processing to let the user know it's working.

In [ ]:
def compute(event):
    with layout.param.update(loading=True):
        time.sleep(3)
        layout.append("Computation complete!")

button = pn.widgets.Button(name="Compute", on_click=compute)
layout = pn.Column("Click below to compute", button)

layout

#### Okay

You can also wrap a `try/finally` to do the same thing.

In [ ]:
def compute(event):
    try:
        layout.loading = True
        time.sleep(3)
        layout.append("Computation complete!")
    finally:
        layout.loading = False

button = pn.widgets.Button(name="Compute", on_click=compute)
layout = pn.Column("Click below to compute", button)

layout

### Manage exceptions gracefully

#### Good

Use:
- `try` block to update values on success
- `except` block to update values on exception
- `finally` block to update values regardless

In [ ]:
import time

def compute(divisor):
    try:
        busy.value = True
        time.sleep(1)
        output = 1 / divisor
        text.value = "Success!"
    except Exception as exc:
        output = "Undefined"
        text.value = f"Error: {exc}"
    finally:
        busy.value = False
    return f"Output: {output}"

busy = pn.widgets.LoadingSpinner(width=10, height=10)
text = pn.widgets.StaticText()

slider = pn.widgets.IntSlider(name="Divisor")
output = pn.bind(compute, slider)

layout = pn.Column(pn.Row(busy, text), slider, output)
layout

### Cache values for speed

#### Good

Wrap the decorator `pn.cache` for automatic handling.

In [ ]:
@pn.cache
def callback(value):
    time.sleep(2)    
    return f"# {value}"

slider = pn.widgets.IntSlider(end=3)
output = pn.bind(callback, slider.param.value_throttled)
pn.Row(slider, output)

#### Okay

Or, manually handle the cache with `pn.state.cache`.

In [ ]:
def callback(value):
    output = pn.state.cache.get(value)
    if output is None:
        time.sleep(2)
        output = f"# {value}"
        pn.state.cache[value] = output
    return output

slider = pn.widgets.IntSlider(end=3)
output = pn.bind(callback, slider.param.value_throttled)
pn.Row(slider, output)

### Preserve axes ranges on update

#### Good

To prevent the plot from resetting to its original axes ranges when zoomed in, simply wrap `hv.DynamicMap`.

In [ ]:
import numpy as np
import holoviews as hv
hv.extension("bokeh")

data = []

def add_point(clicks):
    data.append((np.random.random(), (np.random.random())))
    return hv.Scatter(data)

button = pn.widgets.Button(name="Add point")
plot = hv.DynamicMap(pn.bind(add_point, button.param.clicks))
pn.Column(button, plot)

#### Okay

If you want the object to be completely refreshed, simply drop `hv.DynamicMap`. If it's a long computation, it's good to set `loading_indicator=True`.

In [ ]:
import numpy as np
import holoviews as hv
hv.extension("bokeh")
pn.extension(defer_load=True, loading_indicator=True)

data = []

def add_point(clicks):
    data.append((np.random.random(), (np.random.random())))
    return hv.Scatter(data)

button = pn.widgets.Button(name="Add point")
plot = pn.bind(add_point, button.param.clicks)
pn.Column(button, plot)

### FlexBox instead of Column/Row

#### Good

`pn.FlexBox` automatically moves objects to another row/column, depending on the space available.

In [ ]:
rcolor = lambda: "#%06x" % random.randint(0, 0xFFFFFF)

pn.FlexBox(
    pn.pane.HTML(str(5), styles=dict(background=rcolor()), width=1000, height=100),
    pn.pane.HTML(str(5), styles=dict(background=rcolor()), width=1000, height=100)
)

#### Okay

`pn.Column`/`pn.Row` will overflow if the content is too long/wide.

In [ ]:
rcolor = lambda: "#%06x" % random.randint(0, 0xFFFFFF)

pn.Row(
    pn.pane.HTML(str(5), styles=dict(background=rcolor()), width=1000, height=100),
    pn.pane.HTML(str(5), styles=dict(background=rcolor()), width=1000, height=100)
)

### Reuse objects for efficiency

#### Good

Imagine Panel components as placeholders and use them as such rather than re-creating them on update.

In [ ]:
def randomize(event):
    df_pane.object = pd.DataFrame(np.random.randn(10, 3), columns=list("ABC"))


button = pn.widgets.Button(name="Compute", on_click=randomize)
df_pane = pn.pane.DataFrame()
button.param.trigger("clicks")  # initialize

pn.Column(button, df_pane)

#### Bad

This instantiates the `pn.pane.DataFrame` on every click.

In [ ]:
def randomize(clicks):
    return pn.pane.DataFrame(pd.DataFrame(np.random.randn(10, 3), columns=list("ABC")))


button = pn.widgets.Button(name="Compute")
df_pane = pn.bind(randomize, button.param.clicks)
button.param.trigger("clicks")  # initialize

pn.Column(button, df_pane)